# 4.3 Densely Connected on D2 Labels

In this notebook we will attempt to learn on the decision labels 'D2', which have a 50/50 buy/sell split without hold positions indicated. We hope this will be an easier decision function to learn, rather than the 3 class classification of buy/hold/sell. 

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

In [2]:
y = np.load('./data/prepared/august25screenfixed/numpy_matrices/yD2.npy')

In [3]:
y.shape

(30, 4000)

In [4]:
y_train = y[:, :2000]
y_test = y[:, 2000:]

y_train.shape, y_test.shape

((30, 2000), (30, 2000))

In [5]:
y_train = y_train.reshape([60000, 1])
y_test = y_test.reshape([60000, 1])
y_train.shape, y_test.shape

((60000, 1), (60000, 1))

In [6]:
X_train = np.load('./data/prepared/august25screenfixed/numpy_matrices/Xr3.npy')
X_test = np.load('./data/prepared/august25screenfixed/numpy_matrices/Xe3.npy')

In [7]:
X_train.shape, X_test.shape

((60000, 1, 116, 3), (60000, 1, 116, 3))

In [8]:
np.unique(y_train, return_counts=True)

(array([1., 2.]), array([38654, 21346]))

In [9]:
np.unique(y_test, return_counts=True)

(array([1., 2.]), array([33170, 26830]))

In [10]:
# Undersample to balance classes for training set ?
b = np.where(y_train == 1)[0]
s = np.where(y_train == 2)[0]

bi = np.random.choice(b, size=21000, replace=False)
si = np.random.choice(s, size=21000, replace=False)

ind = np.concatenate([bi,si])
ind.shape

(42000,)

In [11]:
np.unique(y_train[ind], return_counts=True)

(array([1., 2.]), array([21000, 21000]))

In [12]:
X_train[ind].shape

(42000, 1, 116, 3)

In [13]:
y_train = y_train[ind]
X_train = X_train[ind]

In [14]:
X_train.shape, y_train.shape

((42000, 1, 116, 3), (42000, 1))

In [15]:
X_train.shape = (42000, 116, 3)
X_train.shape

(42000, 116, 3)

In [16]:
# decrease test size for runtime and memory concerns
b = np.where(y_test == 1)[0]
s = np.where(y_test == 2)[0]

bi = np.random.choice(b, size=20000, replace=False)
si = np.random.choice(s, size=20000, replace=False)

indt = np.concatenate([bi,si])
indt.shape

y_test = y_test[indt]
X_test = X_test[indt]
y_test.shape, X_test.shape

((40000, 1), (40000, 1, 116, 3))

In [17]:
X_test = X_test.reshape(40000, 116, 3)

In [18]:
np.unique(y_test, return_counts=True)[1]/y_test.shape[0]

array([0.5, 0.5])

In [19]:
np.unique(y_train, return_counts=True),np.unique(y_test, return_counts=True)

((array([1., 2.]), array([21000, 21000])),
 (array([1., 2.]), array([20000, 20000])))

In [20]:
y_train[np.where(y_train == 2)] = 0
y_test[np.where(y_test == 2)] = 0

np.unique(y_train, return_counts=True),np.unique(y_test, return_counts=True)

((array([0., 1.]), array([21000, 21000])),
 (array([0., 1.]), array([20000, 20000])))

#### Attempt 3 columns only

So far, the super simple feed forward network seems to perform just as well as the cnn. 

In [25]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(116, 3)),
    keras.layers.Dense(116, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(2)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50,
         validation_data=(X_test, y_test))

Train on 42000 samples, validate on 40000 samples
Epoch 1/50
42000/42000 [==============================] - 6s 141us/sample - loss: 0.6672 - accuracy: 0.5833 - val_loss: 0.7140 - val_accuracy: 0.4987
Epoch 2/50
42000/42000 [==============================] - 5s 127us/sample - loss: 0.6502 - accuracy: 0.6057 - val_loss: 0.7428 - val_accuracy: 0.4965
Epoch 3/50
42000/42000 [==============================] - 5s 127us/sample - loss: 0.6460 - accuracy: 0.6126 - val_loss: 0.7388 - val_accuracy: 0.4966
Epoch 4/50
42000/42000 [==============================] - 5s 131us/sample - loss: 0.6451 - accuracy: 0.6148 - val_loss: 0.7322 - val_accuracy: 0.4995
Epoch 5/50
42000/42000 [==============================] - 5s 128us/sample - loss: 0.6413 - accuracy: 0.6185 - val_loss: 0.7478 - val_accuracy: 0.5162
Epoch 6/50
42000/42000 [==============================] - 5s 127us/sample - loss: 0.6376 - accuracy: 0.6232 - val_loss: 0.7445 - val_accuracy: 0.5079
Epoch 7/50
42000/42000 [==========================

KeyboardInterrupt: 

In [20]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

predictions = probability_model.predict(X_train)

In [27]:
predictions[4]

array([0.07750326, 0.92002517, 0.00247163], dtype=float32)

In [22]:
np.argmax(predictions)

13971

In [60]:
predictions = probability_model.predict(X_test)

In [61]:
np.argmax(predictions[0])

0

In [62]:
predictions[0]

array([9.8680556e-01, 1.2953308e-02, 2.4112873e-04], dtype=float32)

### implement data augmentation and dropout 